# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [3]:
import pandas as pd


url1 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis%20/Datasets/dim_add_on_categories_cleaned.csv"
url2 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis%20/Datasets/dim_add_ons_cleaned.csv"
url3 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis%20/Datasets/dim_items_cleaned.csv"
url4 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis%20/Datasets/dim_menu_item_add_on_categories_cleaned.csv"
url5 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis%20/Datasets/dim_menu_item_add_ons_cleaned.csv"
url6 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis%20/Datasets/dim_menu_items_cleaned.csv"
url7 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis%20/Datasets/dim_menu_sections_cleaned.csv"
url8 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis%20/Datasets/dim_sections_cleaned.csv"
# FIXED: Removed the extra dot after 'raw'
url9 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis%20/Datasets/dim_users_cleaned.csv"
url10 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis%20/Datasets/menu_engineering_desc_cleaned.csv"

url11 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis%20/Datasets/menu_engineering_dogs.csv"

url12 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis%20/Datasets/menu_engineering_input_items.csv"

url13 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis%20/Datasets/menu_engineering_plowhorse_whatif.csv"
# Loading Data
dim_add_on_categories_cleaned = pd.read_csv(url1)
dim_add_ons_cleaned = pd.read_csv(url2)
dim_items_cleaned = pd.read_csv(url3, low_memory=False)
dim_menu_item_add_on_categories_cleaned = pd.read_csv(url4)
dim_menu_item_add_ons_cleaned = pd.read_csv(url5)
dim_menu_items_cleaned = pd.read_csv(url6)
dim_menu_sections_cleaned = pd.read_csv(url7)
dim_sections_cleaned = pd.read_csv(url8)
dim_users_cleaned = pd.read_csv(url9, low_memory=False)
analysis1  = pd.read_csv(url10)
analysis2  = pd.read_csv(url11)
analysis3  = pd.read_csv(url12)
analysis4  = pd.read_csv(url13)




In [13]:
from mlxtend.preprocessing import TransactionEncoder

transactions = (
   dim_items_cleaned.groupby('purchases')['price']
      .apply(list)
      .tolist()
)

te = TransactionEncoder()
te_array = te.fit(transactions).transform(transactions)

basket = pd.DataFrame(te_array, columns=te.columns_)


In [26]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(
    basket,
    min_support=0.15,
    use_colnames=True
)

print(frequent_itemsets)


    support           itemsets
0  0.235357   frozenset({0.0})
1  0.179269  frozenset({20.0})
2  0.221157  frozenset({25.0})
3  0.177494  frozenset({30.0})
4  0.200923  frozenset({35.0})
5  0.161519  frozenset({40.0})
6  0.165779  frozenset({45.0})


In [11]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(
    frequent_itemsets,
    metric="confidence",
    min_threshold=0.70
)


In [10]:
strong_rules = rules[
    (rules['support'] >= 0.15) &
    (rules['confidence'] >= 0.70)
]

strong_rules = strong_rules.sort_values(
    by=['confidence', 'support'],
    ascending=False
)

print(strong_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


Empty DataFrame
Columns: [antecedents, consequents, support, confidence, lift]
Index: []


In [25]:
import pandas as pd
import numpy as np
from itertools import combinations

# 1. Load Data with exact naming requested
# (Replace the strings with your URLs as defined in your previous snippet)
dim_add_ons_cleaned = pd.read_csv(url2)
dim_items_cleaned = pd.read_csv(url3, low_memory=False)


# We focus on the titles and their category groupings
addons = dim_add_ons_cleaned[['id', 'title', 'category_id']].copy()

#  Link Menu Items → Add-on Categories
# "We focus on items that have available add-ons"
item_addon_cats = dim_items_cleaned[['id', 'add_on_category_ids']].dropna().copy()

# Split the pipe-separated IDs into a list
item_addon_cats['category_id'] = (
    item_addon_cats['add_on_category_ids']
    .astype(str)
    .str.split('|')
)

# Explode the list so each row is a unique Item-Category pair
item_addon_cats = item_addon_cats.explode('category_id')
item_addon_cats['category_id'] = pd.to_numeric(item_addon_cats['category_id'], errors='coerce')
item_addon_cats = item_addon_cats.dropna()

#  Attach Add-on Titles
# This merges the item links with the actual add-on names
item_addons = pd.merge(
    item_addon_cats,
    addons,
    on='category_id',
    how='inner',
    suffixes=('_item_link', '')
)


#  Generate Transactions

# Each "Transaction" is the set of add-ons available to one specific menu item
transactions = (
    item_addons
    .groupby('id_item_link')['title']
    .apply(lambda x: sorted(set(x)))
    .tolist()
)

total_transactions = len(transactions)


#  Calculate Frequencies (Manual Apriori)

item_counts = {}
pair_counts = {}

for transaction in transactions:
    for item in transaction:
        item_counts[item] = item_counts.get(item, 0) + 1
    for pair in combinations(transaction, 2):
        pair = tuple(sorted(pair))
        pair_counts[pair] = pair_counts.get(pair, 0) + 1

# Build Association Rules

# Filter by Support > 15% and Confidence > 70%
rules = []

for (a, b), ab_count in pair_counts.items():
    support_ab = ab_count / total_transactions
    support_a = item_counts[a] / total_transactions
    support_b = item_counts[b] / total_transactions

    # Direction: A → B
    conf_ab = support_ab / support_a
    if support_ab >= 0.15 and conf_ab >= 0.70:
        rules.append({
            'antecedent': a,
            'consequent': b,
            'support': support_ab,
            'confidence': conf_ab,
            'lift': support_ab / (support_a * support_b)
        })

    # Direction: B → A
    conf_ba = support_ab / support_b
    if support_ab >= 0.15 and conf_ba >= 0.70:
        rules.append({
            'antecedent': b,
            'consequent': a,
            'support': support_ab,
            'confidence': conf_ba,
            'lift': support_ab / (support_a * support_b)
        })

#  Final Output & Export

strong_rules = (
    pd.DataFrame(rules)
    .sort_values(['confidence', 'support'], ascending=False)
    .reset_index(drop=True)
)


print("Strong Add-on Associations Found:")
print(strong_rules.head(10))

# Save to CSV
strong_rules.to_csv('strong_rules.csv', index=False)
strong_rules.to_csv('strong_addon_rules.csv', index=False)

Strong Add-on Associations Found:
  antecedent  consequent   support  confidence      lift
0     Pølser       Bacon  0.194567    1.000000  3.296489
1     Pølser      Skinke  0.194567    1.000000  3.394523
2  Tacosauce  Champignon  0.190092    1.000000  2.895784
3  Tacosauce         Løg  0.190092    1.000000  2.883679
4       Majs      Ananas  0.273352    0.998389  2.859284
5     Pølser      Ananas  0.194252    0.998380  2.859260
6   Kødsauce  Champignon  0.302534    0.995025  2.881377
7  Oksefilet  Champignon  0.199168    0.994336  2.879382
8  Oksefilet       Kebab  0.199168    0.994336  2.971024
9  Spaghetti         Løg  0.286020    0.989318  2.852874
